## Example 1a - SENTINEL Hub Data Access using xcube

This notebook shows the various ways to open data cubes from Sentinel Hub (SH) for a given time range, region, and spatial resolution:

1. A **temporarily regular Sentinel-2 cube** with aggregated observations that fall into equal-size time periods;
2. A **temporarily irregular Sentinel-2 cube** which only includes time stamps where there were valid observations and saving the cube to an S3 bucket;
3. A cube using **projected coordinates**.
4. Writing to and reopen from (S3) **object storage**.


To run this Notebook, make sure the SENTINEL Hub / xcube Integration is setup correctly, see [Ex0-DCFS-Setup](./Ex0-DCFS-Setup.ipynb).

In [ ]:
# xcube_sh imports
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube_sh.observers import Observers
from xcube_sh.viewer import ViewerServer

# xcube imports
import xcube
from xcube.core.dsio import write_cube
from xcube.core.maskset import MaskSet
from xcube.core.geom import mask_dataset_by_geometry
from xcube.core.geom import clip_dataset_by_geometry


# Various utilities
import json
import xarray as xr
import shapely.geometry
import IPython.display
import time
import zarr

In [ ]:
%matplotlib inline

---
For this demo, we are focussing on small coastal area near Kiel in Northern Germany (Baltic Sea)

In [ ]:
x1 = 10.00  # degree
y1 = 54.27  # degree
x2 = 11.00  # degree
y2 = 54.60  # degree

bbox = x1, y1, x2, y2

Visualize the bounding box. If you don't see anything, please refer to [Ex0-DCFS-Setup](./Ex0-DCFS-Setup.ipynb).

In [ ]:
IPython.display.GeoJSON(shapely.geometry.box(*bbox).__geo_interface__)

Later in this NB we are going to compute some indexes from bands atmospherically corrected bands B04, B05, B06, B11 of Sentinel-2 (S2L2A)
Our time range covers two and a half month of last year's summer: 2018-05-14 to 2018-07-31

The desired resolution is roughly 20 meters per pixel:

In [ ]:
spatial_res = 0.00018   # = 20.038 meters in degree>

---
### Example 1a1 - Fetch Observations in given Region aggregated into 2-Day Intervals

Sentinel-2 L2A with aggregated observations that fall into equal-size `time_period` of 2 days:

In [ ]:
cube_config = CubeConfig(dataset_name='S2L2A',
                         band_names=['B04', 'B05', 'B06', 'B11', 'SCL', 'CLD'],
                         tile_size=[512, 512],
                         bbox=bbox,
                         spatial_res=spatial_res,
                         time_range=['2018-05-14', '2018-07-31'],
                         time_period='2D')

We define a `request_collector` as an observer for SH requests made, so we can show SH usage stats. This is a developer tool, useful for demonstration purposes too. **Otherwise, this is not needed.**

In [ ]:
request_collector = Observers.request_collector()

Open a data cube:

In [ ]:
cube = open_cube(cube_config, observer=request_collector)

In [ ]:
cube

No requests have been made yet. Requests are made only if data is actually required.

In [ ]:
request_collector.stats

Note, the cube's time coordinates are monotonically increasing and the distance between two time steps is varying:

In [ ]:
cube.time.diff(dim='time').plot.line()

In [ ]:
cube.B04

In [ ]:
cube.B04.sel(time='2018-05-21 10:00:00', method='nearest').plot.imshow(vmin=0, vmax=0.2, cmap='Greys_r', figsize=(16, 10))

Now SentinelHub data requests have been made

In [ ]:
request_collector.stats

xcube mask sets also follow data cube structure

---
### Example 1a2 - Fetch all Observations in given Region

Sentinel-2 L2A which only includes time stamps where there were valid observations for a given region. We use a `time_tolerance` of 30 minutes to decide whether scenes shall be combined:

In [ ]:
cube_config = CubeConfig(dataset_name='S2L2A',
                         band_names=['B04', 'B05', 'B06', 'B11', 'SCL', 'CLD'],
                         tile_size=[1024, 1024],
                         bbox=bbox,
                         spatial_res=spatial_res,
                         time_range=['2018-05-14', '2018-07-31'],                         
                         time_tolerance='30M')

In [ ]:
cube = open_cube(cube_config, observer=request_collector)
cube

In [ ]:
request_collector.stats

In [ ]:
cube.time.diff(dim='time').plot.line()

---
### Example 1a3 - Fetching a Data Cube using another Projection

We now want to fetch a data cube in a given map projection _Pseudo-Mercator_ (EPSG:3857) instead of the Lat/Lon rectangular default projection. This requires setting the `crs` parameter to `'http://www.opengis.net/def/crs/EPSG/0/3857'`. Note that in the output, the spatial coordinate names switched from `lon`, `lat` to `x`, `y`. For other projections, refer to https://docs.sentinel-hub.com/api/latest/#/API/crs.

We need to provide the `bbox` and `spatial_res` parameters in EPSG:3857 units (meters) now, Ljubljana area: 

In [ ]:
x1 = 1545577  # meters
y1 = 5761986  # meters
x2 = 1705367  # meters
y2 = 5857046  # meters

bbox = x1, y1, x2, y2

spatial_res = (x2 - x1) / 512  # meters

Verify we are right:

In [ ]:
geom = shapely.geometry.box(*bbox)

import functools
import pyproj
import shapely.ops

project = functools.partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:3857'),  # source coordinate system (Web Mercator)
    pyproj.Proj(init='epsg:4326'))  # destination coordinate system (WGS-84)

geom_wgs84 = shapely.ops.transform(project, geom)  # apply projection
IPython.display.GeoJSON(geom_wgs84.__geo_interface__)

In [ ]:
cube_config = CubeConfig(dataset_name='S2L2A',
                         band_names=['B04', 'SCL', 'CLD'],
                         tile_size=[512, 512],
                         crs='http://www.opengis.net/def/crs/EPSG/0/3857',
                         bbox=bbox,
                         spatial_res=spatial_res,
                         time_range=['2018-05-01', '2018-05-10'],                         
                         time_period='1D')

In [ ]:
cube = open_cube(cube_config)
cube

In [ ]:
cube.B04.isel(time=5).plot.imshow(vmin=0, vmax=0.18, cmap='Greys_r', figsize=(16, 10))

---
### Example 1a4 - Writing to and Reopening from S3 Object Storage

We now want to write the cube from Example 3 to AWS S3 and then re-open it.

Writing to AWS usually requires credentials to be configured. There are three possible ways to do so:

1. Use the AWS CLI tool `aws configure` (see [AWS docs](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html));
2. Set AWS environment variables `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY`;
3. Pass credentials as parameters `aws_access_key_id` and `aws_secret_access_key` in `s3_client_kwargs` to the xcube `open_cube` and `write_cube` functions.

For other object storage providers, the AWS settings usually work to. You just need to pass another `endpoint_url` in `s3_client_kwargs`, see below.

The xcube `open_cube` and `write_cube` functions have a path parameter that is either absolute `{endpoint_url}/{bucket_name}/{cube_name}` or relative `{bucket_name}/{cube_name}`. In the relative case, `endpoint_url` may be provided in `s3_client_kwargs`. If it is not given, it defaults to the well-known AWS S3 endpoint.

In [29]:
cube_path = 'https://s3.eu-central-1.amazonaws.com/xcube-examples/S2L2A-B04-Test-01.zarr'

Uncomment the code below to write the cube to your own s3 bucket using pre-configured credentials: 

In [30]:
#%%time
#write_cube(cube, output_path=cube_path)

Wall time: 6.85 s


<xarray.Dataset>
Dimensions:    (bnds: 2, time: 10, x: 512, y: 305)
Coordinates:
  * time       (time) datetime64[ns] 2018-05-01T12:00:00 ... 2018-05-10T12:00:00
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(10, 2), meta=np.ndarray>
  * x          (x) float64 1.546e+06 1.546e+06 1.546e+06 ... 1.705e+06 1.705e+06
  * y          (y) float64 5.857e+06 5.857e+06 5.856e+06 ... 5.762e+06 5.762e+06
Dimensions without coordinates: bnds
Data variables:
    B04        (time, y, x) float32 dask.array<chunksize=(1, 305, 512), meta=np.ndarray>
    CLD        (time, y, x) uint8 dask.array<chunksize=(1, 305, 512), meta=np.ndarray>
    SCL        (time, y, x) uint8 dask.array<chunksize=(1, 305, 512), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    title:                     S2L2A Data Cube Subset
    history:                   [{'program': 'xcube_sh.chunkstore.SentinelHubC...
    date_created:              2020-12-09T13:25:28.581632
    processing_level:          L2A
    time_coverage_start:       2018-05-01T00:00:00+00:00
    time_coverage_end:         2018-05-11T00:00:00+00:00
    time_coverage_duration:    P10DT0H0M0S
    time_coverage_resolution:  P1DT0H0M0S

In [31]:
# Note, we can also use s3_client_kwargs to pass the credentials directly or use another endpoint URL or region (all fields are optional):
# If endpoint_url is used then the given path should be relative and starting with the bucket name forllowed by the cube name.
# s3_client_kwargs = dict(aws_access_key_id='my_key', aws_secret_access_key='my_secret', endpoint_url='my_url', region_name='my_region')
# write_cube(cube, output_path=cube_path, s3_client_kwargs=s3_client_kwargs)

In [32]:
request_collector.stats

Number of requests:,44
Request duration min:,469.19 ms
Request duration max:,1169.28 ms
Request duration median:,773.98 ms
Request duration mean:,768.46 ms
Request duration std:,170.23 ms


The s3 bucket of the above specified `cube_path` is publicly readable, so you can have a look at the cube with the following line:

In [35]:
cube_from_bucket = xcube.core.dsio.open_cube(cube_path, s3_kwargs=dict(anon=True))

In [36]:
cube_from_bucket

<xarray.Dataset>
Dimensions:    (bnds: 2, time: 10, x: 512, y: 305)
Coordinates:
  * time       (time) datetime64[ns] 2018-05-01T12:00:00 ... 2018-05-10T12:00:00
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(10, 2), meta=np.ndarray>
  * x          (x) float64 1.546e+06 1.546e+06 1.546e+06 ... 1.705e+06 1.705e+06
  * y          (y) float64 5.857e+06 5.857e+06 5.856e+06 ... 5.762e+06 5.762e+06
Dimensions without coordinates: bnds
Data variables:
    B04        (time, y, x) float32 dask.array<chunksize=(1, 305, 512), meta=np.ndarray>
    CLD        (time, y, x) uint8 dask.array<chunksize=(1, 305, 512), meta=np.ndarray>
    SCL        (time, y, x) uint8 dask.array<chunksize=(1, 305, 512), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    date_created:              2020-12-09T13:25:28.581632
    history:                   [{'cube_config': {'band_names': ['B04', 'SCL',...
    processing_level:          L2A
    time_coverage_duration:    P10DT0H0M0S
    time_coverage_end:         2018-05-11T00:00:00+00:00
    time_coverage_resolution:  P1DT0H0M0S
    time_coverage_start:       2018-05-01T00:00:00+00:00
    title:                     S2L2A Data Cube Subset